In [8]:
pip install --upgrade opencv-python --user

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import time

In [3]:
# Захватыаем вебку
cap = cv2.VideoCapture(0)

# Проверяем, открылось ли видео
if not cap.isOpened():
    print('Video is not oppened')
    
# Задаём параметры видео (если переборщить, то выведется максимальный размер видео)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    

# бесконечный цикл обработки кадров
while True:
    # Считываем кадр
    rat, frame = cap.read()
    
    # Если картинки нет (rat = False), то выходим из цикла
    if not rat:
        print("Error")
        break
    
    # Выводим кадр
    cv2.imshow('Web', frame)
    
    # 'q' - кнопка для прекращения программы
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [19]:
def convert_to_gray(frame):
    # Конвертируем кадр в оттенки серого
    grey_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return grey_frame

In [20]:
def display_green_objects(frame):
    '''
    Функцияя, генерирующая картинку с изображением только зелёных пердметов (Картинка ЧБ)
    '''
    
    # Конвертируем в HSV, так легче извлекать нуджный цвет пикселей
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # указываем границы цвета по HSV формату
    lower_color = np.array([65, 77, 35])
    upper_color = np.array([100, 255, 190])
    
    # Генерируем маску кадра с учётом границ
    green_mask = cv2.inRange(hsv_frame, lower_color, upper_color)
    
    # избавление от артифактов
    kernel = np.ones((15, 15), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    
    # присоеденяем получившуюся картинку снизу к уже имеющейся
    combined_frame = np.vstack((frame, np.repeat(green_mask[:, :, np.newaxis], 3, axis=2)))
    return combined_frame

In [21]:
class FPS_Counter:
    def __init__(self, calc_times_perion_N_frames):
        self.calc_times_perion_N_frames = calc_times_perion_N_frames
        self.buffer = []
        
    def calc_FPS(self):
        time_buffer_is_full = len(self.buffer) == self.calc_times_perion_N_frames
        t = time.time()
        
        self.buffer.append(t)
        
        if time_buffer_is_full:
            self.buffer.pop(0)
            fps = len(self.buffer)/(self.buffer[-1] - self.buffer[0])
            return np.round(fps, 2)
        else:
            return 0.0

In [9]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video is not oppened')
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    

fps_couner = FPS_Counter(50)

while True:
    rat, frame = cap.read()
    
    if not rat:
        print("Error")
        break
    
    # Конвертируем в серый
#     frame = convert_to_gray(frame)
    
    # Добавляем картинку снизу только с зелёными объектами
    frame = display_green_objects(frame)
    
    frame = convert_to_gray(frame)
    
    # Изменяем размер итогового фрейма
#     scale = 1
#     frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    
    fps_count = fps_couner.calc_FPS()
    text = f'FPS: {fps_count}'
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text, 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness)
    
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame, 
        text=text, 
        org=(5,20), 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness, 
        color=(255, 255, 255))
    
    cv2.imshow('Web', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

# YOLO

In [3]:
!pip install patched-yolo-infer

  Obtaining dependency information for patched-yolo-infer from https://files.pythonhosted.org/packages/e5/66/b049e216c5074465065c0ae05e18cef2b490fe6dffb19b60a312518db4d4/patched_yolo_infer-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/74/72/7365c92cca8a00fbd5ced77274cd4f0c63769b87ae7966ee7419935b78b3/ultralytics-8.2.100-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/5a/6a/775b93d6888c31f1f1fc457e4f5cc89f0984412d5dcdef792b8f2aa6e812/torch-2.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/f8/69/dc769cf54df8e828c0b8957b4521f35178f5bd4cc5b8fbe8a37ffd89a27c/torchvision-0.19.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for ultralytics-thop>=2.0.0 from https://files.pythonhosted.org/packages/22/4b/126ba8e757f83fb735d44

In [2]:
from ultralytics import YOLO
from patched_yolo_infer import visualize_results_usual_yolo_inference

In [3]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('Video is not oppened')
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    


rat, frame = cap.read()

In [33]:
imgsize=640
conf = 0.5
iou = 0.7

model = YOLO('yolov8m-seg.pt')

visualize_results_usual_yolo_inference(
    frame, 
    model,
    imgsize,
    conf,
    iou,
    segment=True,
    delta_colors=3,
    thickness=4,
    font_scale=1.5,
    show_boxes=False,
    fill_mask=True,
    alpha=0.2,
    show_confidences=True)

In [22]:
imgsize=640
conf = 0.5
iou = 0.7

model = YOLO('yolov8m-seg.pt')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video is not oppened')
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    

fps_couner = FPS_Counter(50)

while True:
    rat, frame = cap.read()
    
    if not rat:
        print("Error")
        break
    
    # Конвертируем в серый
#     frame = convert_to_gray(frame)
    
    # Добавляем картинку снизу только с зелёными объектами
#     frame = display_green_objects(frame)
        
    # Изменяем размер итогового фрейма
#     scale = 1
#     frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    
    frame = visualize_results_usual_yolo_inference(
            frame, 
            model,
            imgsize,
            conf,
            iou,
            segment=True,
            delta_colors=3,
            thickness=4,
            font_scale=1.5,
            show_boxes=False,
            fill_mask=True,
            alpha=0.2,
            show_confidences=True,
            return_image_array=True)
    
    
    fps_count = fps_couner.calc_FPS()
    text = f'FPS: {fps_count}'
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text, 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness)
    
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame, 
        text=text, 
        org=(5,20), 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness, 
        color=(255, 255, 255))
    
    cv2.imshow('Web', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [24]:
imgsize=640
conf = 0.5
iou = 0.7

model = YOLO('yolov8n.pt')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video is not oppened')
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    

fps_couner = FPS_Counter(50)

while True:
    rat, frame = cap.read()
    
    if not rat:
        print("Error")
        break
    
    # Конвертируем в серый
#     frame = convert_to_gray(frame)
    
    # Добавляем картинку снизу только с зелёными объектами
#     frame = display_green_objects(frame)
        
    # Изменяем размер итогового фрейма
#     scale = 1
#     frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    
    frame = visualize_results_usual_yolo_inference(
            frame, 
            model,
            imgsize,
            conf,
            iou,
            segment=False,
            delta_colors=3,
            thickness=4,
            font_scale=1.5,
            show_boxes=True,
            fill_mask=True,
            alpha=0.2,
            show_confidences=True,
            return_image_array=True)
    
    
    fps_count = fps_couner.calc_FPS()
    text = f'FPS: {fps_count}'
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text, 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness)
    
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame, 
        text=text, 
        org=(5,20), 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness, 
        color=(255, 255, 255))
    
    cv2.imshow('Web', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:00<00:00, 9.49MB/s]


In [37]:
imgsize=640
conf = 0.5
iou = 0.7

model = YOLO('yolov8n.pt')

cap = cv2.VideoCapture('video.mp4')
if not cap.isOpened():
    print('Video is not oppened')
    
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)    
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)    

fps_couner = FPS_Counter(10)


fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))


while True:
    rat, frame = cap.read()
    
    if not rat:
        print("Error")
        break
    
    # Конвертируем в серый
#     frame = convert_to_gray(frame)
    
    # Добавляем картинку снизу только с зелёными объектами
#     frame = display_green_objects(frame)
        
    # Изменяем размер итогового фрейма
    frame = cv2.resize(frame, (width, height))
    
    # Не работает
#     scale = 0.5
#     frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
     
    frame = visualize_results_usual_yolo_inference(
            frame, 
            model,
            imgsize,
            conf,
            iou,
            segment=False,
            delta_colors=3,
            thickness=4,
            font_scale=1.5,
            show_boxes=True,
            fill_mask=True,
            alpha=0.2,
            show_confidences=True,
            return_image_array=True)
    
    
    fps_count = fps_couner.calc_FPS()
    text = f'FPS: {fps_count}'
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text, 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness)
    
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame, 
        text=text, 
        org=(5,20), 
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness, 
        color=(255, 255, 255))
    
    
    out.write(frame)
    cv2.imshow('Web', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
out.release()
cv2.destroyAllWindows()